In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import numpy as np
from PIL import Image
# Load PoseNet model
def load_posenet_model():
    model_url = 'https://tfhub.dev/google/movenet/singlepose/lightning/4'
    model = hub.load(model_url)
    return model

posenet_model = load_posenet_model()

In [ ]:
def extract_keypoints_using_posenet(image_tensor, model):
    # Check if the image_tensor needs resizing to match the model input requirements
    # Assuming the required size is 192x192
    print("Image tensor shape before resizing:", image_tensor.shape)
    image_tensor = tf.image.resize_with_pad(image_tensor, 192, 192)
    image_tensor = image_tensor * 255  # Scale from [0, 1] to [0, 255]
    image_tensor = tf.cast(image_tensor, tf.int32)  # Cast to int32
    

    # Ensure there's a batch dimension
    image_tensor = tf.expand_dims(image_tensor, axis=0)

    # Use the model to get the keypoints
    infer = model.signatures['serving_default']
    results = infer(image_tensor)
    keypoints = results['output_0'].numpy()[0]  # Adjust indexing based on your model's output format

    return keypoints


In [ ]:
import tensorflow as tf


def load_and_preprocess_image(image_path):
    # Load the image using PIL
    image = Image.open(image_path)
    
    # Convert the image to RGB if it's not in RGB format
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Resize the image to the required dimensions
    image = image.resize((192, 192))
    
    # Convert the image to a numpy array
    image_array = np.array(image)
    
    # Ensure the image has the expected shape
    if len(image_array.shape) != 3 or image_array.shape[2] != 3:
        raise ValueError("Invalid image shape. Expected shape: (height, width, 3)")
    
    # Normalize the pixel values to the range [0, 1]
    image_array = image_array / 255.0
    
    # Add batch dimension to the image tensor
    image_tensor = tf.expand_dims(image_array, 0)
    
    return image_tensor

def process_directory(directory, model, keypoints_dict):
    """
    Recursively process each folder and its subfolders to extract keypoints from yoga pose images.

    :param directory: The current directory to process.
    :param model: The loaded PoseNet model for extracting keypoints.
    :param keypoints_dict: A dictionary to store keypoints with yoga pose names as keys.
    """
    # Iterate over all files and directories in the current directory
    for entry in os.listdir(directory):
        entry_path = os.path.join(directory, entry)
        
        # If entry is a directory, recursively process it
        if os.path.isdir(entry_path):
            process_directory(entry_path, model, keypoints_dict)
        
        # If entry is an image file, process it
        elif entry.lower().endswith(('.jpg', '.png', '.jpeg')):
            # Extract yoga pose name from the directory name
            yoga_pose_name = os.path.basename(os.path.dirname(entry_path))
            
            # Load and preprocess the image
            try:
                image_tensor = load_and_preprocess_image(entry_path)
                
                # Extract keypoints from the image
                keypoints = extract_keypoints_using_posenet(image_tensor, model)
                
                # Initialize the list for this pose if it doesn't exist
                if yoga_pose_name not in keypoints_dict:
                    keypoints_dict[yoga_pose_name] = []
                
                # Append the extracted keypoints for this pose
                keypoints_dict[yoga_pose_name].append(keypoints)
            except ValueError as e:
                print(f"Error processing image {entry_path}: {e}")

# Initialize an empty dictionary to store the keypoints
reference_poses_keypoints = {}

# Start processing from the root data directory
root_data_directory = 'DATASET\data'  # Update this path to your dataset location
process_directory(root_data_directory, posenet_model, reference_poses_keypoints)


In [ ]:
import json

# Convert numpy arrays to lists for JSON serialization
for pose, keypoints in reference_poses_keypoints.items():
    reference_poses_keypoints[pose] = [kp.tolist() for kp in keypoints]

with open('reference_poses_keypoints.json', 'w') as f:
    json.dump(reference_poses_keypoints, f)


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import json
import math
import os
from PIL import Image

# Load YOLO model
def load_yolo_model():
    net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
    layer_names = net.getLayerNames()
    
    # Adjusted line: Directly use elements from getUnconnectedOutLayers() as indices
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

    return net, output_layers


# Detect and crop the person from an image
def detect_and_crop_person(image_path, net, output_layers):
    image = cv2.imread(image_path)
    (H, W) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)
    
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Detected class is person
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                cropped_image = image[y:y+height, x:x+width]
                return cropped_image
    return None

# Load PoseNet model for pose estimation
def load_posenet_model():
    model_url = 'https://tfhub.dev/google/movenet/singlepose/lightning/4'
    model = hub.load(model_url)
    return model

# Preprocess image for PoseNet
def preprocess_image_for_posenet(image):
    image = tf.image.resize(image, (192, 192))  # Resize the image
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    # Assuming 'processed_image' is your input tensor and it's currently a float tensor
    processed_image = tf.cast(image, tf.int32)  # Cast to int32 if the model expects int32 inputs
    return image

def preprocess_image_for_posenet(image):
    image = tf.image.resize(image, (192, 192))  # Resize the image
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    image = tf.cast(image, tf.int32)  # Cast to int32 to match the model's expected input type
    return image


# Calculate angle using law of cosines
def calculate_angle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle
'''
# Provide pose correction feedback using angles
def provide_pose_correction_feedback_using_angles(user_keypoints, reference_keypoints):
    feedback = []
    angle_keypoints_indices = [(5, 6, 7), (1, 2, 3)]  # Update with actual indices as needed

    for indices in angle_keypoints_indices:
        user_angle = calculate_angle(user_keypoints[indices[0]], user_keypoints[indices[1]], user_keypoints[indices[2]])
        ref_angle = calculate_angle(reference_keypoints[indices[0]], reference_keypoints[indices[1]], reference_keypoints[indices[2]])
        if abs(user_angle - ref_angle) > 10:  # Define your threshold
            feedback.append(f"Adjust your position; the angle formed by keypoints {indices} is off by {abs(user_angle - ref_angle):.2f} degrees.")

    return feedback

# Main function to handle user uploaded image
def handle_user_uploaded_image(user_image_path, intended_pose_name):
    yolo_net, yolo_layers = load_yolo_model()
    user_image = detect_and_crop_person(user_image_path, yolo_net, yolo_layers)
    if user_image is not None:
        posenet_model = load_posenet_model()
        user_image_tf = tf.convert_to_tensor(user_image, dtype=tf.float32)
        user_image_tf = tf.image.convert_image_dtype(user_image_tf, tf.float32)
        user_image_tf = tf.image.resize_with_pad(user_image_tf, 192, 192)
        user_image_tf = tf.expand_dims(user_image_tf, axis=0)

        user_keypoints = estimate_pose_with_posenet(posenet_model, user_image_tf)[0]

        # Load reference keypoints from a JSON file
        with open('reference_poses_keypoints.json', 'r') as f:
            reference_poses_keypoints = json.load(f)

        reference_keypoints = reference_poses_keypoints.get(intended_pose_name, [])

        if reference_keypoints:
            correction_feedback = provide_pose_correction_feedback_using_angles(user_keypoints, reference_keypoints[0])
            for line in correction_feedback:
                print(line)
        else:
            print(f"No reference keypoints available for pose: {intended_pose_name}")
    else:
        print("No person detected in the image.")

# Example usage
user_image_path = r"DATASET\data\anantasana\2-0.png"
intended_pose_name = "anantasana"
handle_user_uploaded_image(user_image_path, intended_pose_name)
'''

In [ ]:

POSE_ANGLES_KEYPOINTS = {
    'Parighasana': [  # Gate Pose
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder
        (11, 12, 24)  # Angle between left hip, right hip, and right knee (assuming 24 is an extended right leg keypoint, if available)
    ],
    'Salabhasana': [  # Locust Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (lifted leg position)
        (12, 14, 16)  # Angle between right hip, right knee, and right ankle (lifted leg position)
    ],
    'Vajrasana': [  # Thunderbolt Pose
        (5, 11, 12),  # Angle between right shoulder, left hip, and right hip (to check upright torso)
        (13, 11, 15)  # Angle between left knee, left hip, and left ankle (to check folded legs alignment)
    ],
    'Parsva Bakasana': [  # Side Crow Pose
        (5, 7, 9),  # Angle between right shoulder, right elbow, and right wrist (arm balance alignment)
        (6, 8, 10)  # Angle between left shoulder, left elbow, and left wrist (arm balance alignment)
    ],
    'Ardha Pincha Mayurasana': [  # Dolphin Pose
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder (to check shoulder alignment)
        (11, 23, 12)  # Angle between left hip, tailbone (if available), and right hip (to check hip alignment)
    ],
    'Ananda Balasana': [  # Happy Baby Pose
        (11, 15, 13),  # Angle between left hip, left ankle, and left knee (to check leg's open position)
        (12, 16, 14)  # Angle between right hip, right ankle, and right knee (to check leg's open position)
    ],
    'Makarasana': [  # Crocodile Pose
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder (to ensure relaxed shoulders)
        (11, 23, 12)  # Angle between left hip, tailbone (if available), and right hip (to ensure aligned hips)
    ],
    'Lolasana': [  # Pendant Pose
        (7, 5, 6),  # Angle between left elbow, right shoulder, and left shoulder (to check lifted body)
        (8, 6, 5)  # Angle between right elbow, left shoulder, and right shoulder (to check lifted body)
    ],
    'Janu Sirsasana': [  # Head-to-Knee Forward Bend
        (12, 14, 16),  # Angle between right hip, right knee, and right ankle (extended leg)
        (11, 13, 15)  # Angle between left hip, left knee, and left ankle (bent leg)
    ],
    'Eka Pada Koundinyanasana II': [  # Pose dedicated to the sage Koundinya
        (7, 5, 11),  # Angle between left elbow, right shoulder, and left hip (arm balance alignment)
        (8, 6, 12)  # Angle between right elbow, left shoulder, and right hip (arm balance alignment)
    ],
    'Ardha Chandrasana': [  # Half Moon Pose
        (11, 23, 15),  # Angle between left hip, center of the torso, and left ankle (standing leg to raised leg)
        (5, 1, 6)  # Angle between right shoulder, neck, and left shoulder (arm alignment)
    ],
    'Baddha Konasana': [  # Bound Angle Pose
        (11, 23, 15),  # Angle between left hip, center of the torso, and left ankle (leg's open position)
        (12, 23, 16)  # Angle between right hip, center of the torso, and right ankle (leg's open position)
    ],
    'Chakravakasana': [  # Cat-Cow Stretch (considering Cat pose)
        (1, 11, 15),  # Angle between neck, left hip, and left ankle (to check back roundness)
        (1, 12, 16)  # Angle between neck, right hip, and right ankle (to check back roundness)
    ],
    'Agnistambhasana': [  # Fire Log Pose
        (11, 23, 15),  # Angle between left hip, center of the torso, and left ankle (to check shin stack alignment)
        (12, 23, 16)  # Angle between right hip, center of the torso, and right ankle (to check shin stack alignment)
    ],
    'Marichyasana I': [  # Marichi's Pose I
        (12, 14, 16),  # Angle between right hip, right knee, and right ankle (seated leg's position)
        (11, 15, 13)  # Angle between left hip, left ankle, and left knee (to check the bound position if applicable)
    ],
    'Camatkarasana': [  # Wild Thing
        (5, 11, 15),  # Angle between right shoulder, left hip, and left ankle (to check the lift and open chest)
        (6, 12, 16)  # Angle between left shoulder, right hip, and right ankle (to check the grounding of the foot)
    ],
    'Prasarita Padottanasana': [  # Wide-Legged Forward Bend
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to check hip alignment)
        (23, 11, 15),  # Angle between center of the torso, left hip, and left ankle (to check forward fold depth)
    ],
    'Ardha Matsyendrasana': [  # Half Lord of the Fishes Pose
        (11, 1, 12),  # Angle between left hip, neck, and right hip (to check the twist)
        (5, 1, 6)  # Angle between right shoulder, neck, and left shoulder (to check shoulder alignment)
    ],
    'Parivrtta Trikonasana': [  # Revolved Triangle Pose
        (5, 11, 15),  # Angle between right shoulder, left hip, and left ankle (to check the twist and alignment)
        (6, 12, 16)  # Angle between left shoulder, right hip, and right ankle (to support the twist)
    ],
    'Virabhadrasana II': [  # Warrior II Pose
        (5, 11, 13),  # Angle between right shoulder, left hip, and left knee (front leg alignment)
        (6, 12, 14)  # Angle between left shoulder, right hip, and right knee (back leg alignment)
    ],
    'Salamba Sirsasana': [  # Supported Headstand
        (0, 1, 2),  # Angle between nose, neck, and top of the head (to check head and spine alignment)
        (11, 23, 12)  # Angle between left hip, center of the torso, and right hip (to check leg and hip alignment)
    ],
    'Uttana Shishosana': [  # Extended Puppy Pose
        (1, 11, 15),  # Angle between neck, left hip, and left ankle (to check back and arm stretch)
        (1, 12, 16)  # Angle between neck, right hip, and right ankle (to check back and arm stretch)
    ],
    'Makara Adho Mukha Svanasana': [  # Dolphin Plank Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure straight line in plank)
        (5, 7, 9)  # Angle between right shoulder, right elbow, and right wrist (to check arm alignment)
    ],
    'Adho Mukha Vriksasana': [  # Handstand
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder (to check shoulder alignment)
        (11, 23, 12)  # Angle between left hip, center of the torso, and right hip (to ensure hips are over shoulders)
    ],
    'Utthita Ashwa Sanchalanasana': [  # High Lunge
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (front leg alignment)
        (12, 24, 14)  # Angle between right hip, center of the torso, and right knee (back leg and torso alignment)
    ],
    'Supta Virasana': [  # Reclining Hero Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to check thigh and shin alignment)
        (12, 14, 16)  # Angle between right hip, right knee, and right ankle (to check thigh and shin alignment)
    ],
    'Virasana': [  # Hero Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are centered and grounded)
        (13, 23, 14)  # Angle between left knee, center of the torso, and right knee (to check knee alignment)
    ],
    'Hanumanasana': [  # Monkey Pose or Splits
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (front leg alignment)
        (12, 14, 16)  # Angle between right hip, right knee, and right ankle (back leg alignment)
    ],
    'Virabhadrasana I': [  # Warrior I Pose
        (5, 11, 13),  # Angle between right shoulder, left hip, and left knee (to check alignment of front leg and upper body)
        (12, 14, 16)  # Angle between right hip, right knee, and right ankle (back leg alignment)
    ],
    'Gomukhasana': [  # Cow Face Pose
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder (to check shoulder alignment and stretch)
        (11, 23, 12)  # Angle between left hip, center of the torso, and right hip (to ensure hips are evenly grounded)
    ],
    'Simhasana': [  # Lion Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to check seated posture and spine alignment)
        (5, 1, 6)  # Angle between right shoulder, neck, and left shoulder (to ensure shoulders are broad and open)
    ],
    'Ganda Bherundasana': [  # Formidable Face Pose
        (5, 1, 6),  # Angle between right shoulder, neck, and left shoulder (to check shoulder engagement and lift)
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to check leg extension and hip rotation)
    ],
    'Dandasana': [  # Staff Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure straight seated posture)
        (5, 1, 6)  # Angle between right shoulder, neck, and left shoulder (to ensure shoulders are aligned with hips)
    ], 
    'Urdhva Dhanurasana': [  # Upward Bow (Wheel) Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to ensure full extension)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (to ensure full extension)
    ],
    'Ardha Bhekasana': [  # Half Frog Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to check knee and ankle alignment)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (for the extended leg)
    ],
    'Bakasana': [  # Crane (Crow) Pose
        (5, 7, 9),    # Angle between right shoulder, right elbow, and right wrist (to ensure proper arm bend)
        (6, 8, 10)    # Angle between left shoulder, left elbow, and left wrist (to ensure proper arm bend)
    ],
    'Purvottanasana': [  # Upward Plank Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are lifted)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to check arm extension)
    ],
    'Phalakasana': [  # Plank Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure a straight line)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to ensure arms are straight)
    ],
    'Salamba Bhujangasana': [  # Sphinx Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure engagement of the lower back)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (for forearm placement)
    ],
    'Kurmasana': [  # Tortoise Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are grounded)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to check arm extension under legs)
    ],
    'Utthita Hasta Padangustasana': [  # Extended Hand-To-Big-Toe Pose
        (5, 11, 15),  # Angle between right shoulder, left hip, and left ankle (for standing leg and arm extension)
        (6, 12, 16)   # Angle between left shoulder, right hip, and right ankle (for lifted leg alignment)
    ],
    'Eka Pada Koundinyanasana I': [  # Pose Dedicated to the Sage Koundinya I
        (5, 7, 9),    # Angle between right shoulder, right elbow, and right wrist (to ensure proper arm bend)
        (6, 8, 10)    # Angle between left shoulder, left elbow, and left wrist (to ensure proper arm bend)
    ],
    'Virabhadrasana III': [  # Warrior III Pose
        (1, 11, 15),  # Angle between neck, left hip, and left ankle (for balance and alignment)
        (1, 12, 16)   # Angle between neck, right hip, and right ankle (for balance and alignment)
    ],
    'Bitilasana': [  # Cow Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure the curve of the spine)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (to check shoulder openness)
    ],
    'Parsvottanasana': [  # Intense Side Stretch Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are squared)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (for upper body alignment)
    ],
    'Anjaneyasana': [  # Low Lunge
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (front leg alignment)
        (12, 24, 14)   # Angle between right hip, center of the torso, and right knee (back leg alignment)
    ],
    'Sukhasana': [  # Easy Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure seated posture)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (to ensure spine is elongated)
    ],
    'Dhanurasana': [  # Bow Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to ensure the bend of the legs)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (to ensure the bend of the legs)
    ],
    'Urdhva Dhanurasana': [  # Upward Bow (Wheel) Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to ensure full extension)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (to ensure full extension)
    ],
    'Ardha Bhekasana': [  # Half Frog Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to check knee and ankle alignment)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (for the extended leg)
    ],
    'Bakasana': [  # Crane (Crow) Pose
        (5, 7, 9),    # Angle between right shoulder, right elbow, and right wrist (to ensure proper arm bend)
        (6, 8, 10)    # Angle between left shoulder, left elbow, and left wrist (to ensure proper arm bend)
    ],
    'Purvottanasana': [  # Upward Plank Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are lifted)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to check arm extension)
    ],
    'Phalakasana': [  # Plank Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure a straight line)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to ensure arms are straight)
    ],
    'Salamba Bhujangasana': [  # Sphinx Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure engagement of the lower back)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (for forearm placement)
    ],
    'Kurmasana': [  # Tortoise Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are grounded)
        (5, 7, 9)     # Angle between right shoulder, right elbow, and right wrist (to check arm extension under legs)
    ],
    'Utthita Hasta Padangustasana': [  # Extended Hand-To-Big-Toe Pose
        (5, 11, 15),  # Angle between right shoulder, left hip, and left ankle (for standing leg and arm extension)
        (6, 12, 16)   # Angle between left shoulder, right hip, and right ankle (for lifted leg alignment)
    ],
    'Eka Pada Koundinyanasana I': [  # Pose Dedicated to the Sage Koundinya I
        (5, 7, 9),    # Angle between right shoulder, right elbow, and right wrist (to ensure proper arm bend)
        (6, 8, 10)    # Angle between left shoulder, left elbow, and left wrist (to ensure proper arm bend)
    ],
    'Virabhadrasana III': [  # Warrior III Pose
        (1, 11, 15),  # Angle between neck, left hip, and left ankle (for balance and alignment)
        (1, 12, 16)   # Angle between neck, right hip, and right ankle (for balance and alignment)
    ],
    'Bitilasana': [  # Cow Pose
        (1, 11, 12),  # Angle between neck, left hip, and right hip (to ensure the curve of the spine)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (to check shoulder openness)
    ],
    'Parsvottanasana': [  # Intense Side Stretch Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure hips are squared)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (for upper body alignment)
    ],
    'Anjaneyasana': [  # Low Lunge
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (front leg alignment)
        (12, 24, 14)   # Angle between right hip, center of the torso, and right knee (back leg alignment)
    ],
    'Sukhasana': [  # Easy Pose
        (11, 23, 12),  # Angle between left hip, center of the torso, and right hip (to ensure seated posture)
        (5, 1, 6)     # Angle between right shoulder, neck, and left shoulder (to ensure spine is elongated)
    ],
    'Dhanurasana': [  # Bow Pose
        (11, 13, 15),  # Angle between left hip, left knee, and left ankle (to ensure the bend of the legs)
        (12, 14, 16)   # Angle between right hip, right knee, and right ankle (to ensure the bend of the legs)
    ],
    
    'Parivrtta Parsvakonasana': [  # Revolved Side Angle Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (for twist and alignment)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (for balance and stability)
    ],
    'Marichyasana III': [  # Marichi's Pose III
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (for twist alignment)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to check the seated posture)
    ],
    'Adho Mukha Svanasana': [  # Downward-Facing Dog
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to ensure proper hip elevation)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to ensure proper hip elevation)
    ],
    'Ashtanga Namaskara': [  # Eight-Limbed Pose
        (5, 7, 9),    # RightShoulder, rightElbow, rightWrist (to ensure elbows are bent correctly)
        (6, 8, 10)    # LeftShoulder, leftElbow, leftWrist (to ensure elbows are bent correctly)
    ],
    'Anantasana': [  # Side-Reclining Leg Lift
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to check leg lift and extension)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to maintain balance on the lying side)
    ],
    'Padangusthasana': [  # Big Toe Pose
        (5, 11, 15),  # RightShoulder, leftHip, leftAnkle (to ensure forward bend and leg straightness)
        (6, 12, 16)   # LeftShoulder, rightHip, rightAnkle (to ensure forward bend and leg straightness)
    ],
    'Ardha Uttanasana': [  # Standing Half Forward Bend
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to ensure a flat back)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to ensure a flat back)
    ],
    'Urdhva Hastasana': [  # Upward Salute
        (5, 1, 6),    # RightShoulder, neck, leftShoulder (to ensure arms are straight and aligned with ears)
        (11, 23, 12)  # LeftHip, center torso, rightHip (to ensure lower body stability)
    ],
    'Yoganidrasana': [  # Yogic Sleep Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (for the leg over the shoulder)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (for the leg over the shoulder)
    ],
    'Chaturanga Dandasana': [  # Four-Limbed Staff Pose
        (5, 7, 9),    # RightShoulder, rightElbow, rightWrist (to ensure proper 90-degree angle at elbows)
        (6, 8, 10)    # LeftShoulder, leftElbow, leftWrist (to ensure proper 90-degree angle at elbows)
    ],
    'Marjaryasana': [  # Cat Pose
        (1, 11, 12),  # Neck, leftHip, rightHip (to ensure the rounding of the back)
        (5, 1, 6)     # RightShoulder, neck, leftShoulder (to ensure shoulder blade protraction)
    ],
    'Natarajasana': [  # Lord of the Dance Pose
        (5, 11, 15),  # RightShoulder, leftHip, leftAnkle (to ensure the back leg's lift and extension)
        (6, 12, 16)   # LeftShoulder, rightHip, rightAnkle (to ensure balance and front body stretch)
    ],
    'Mayurasana': [  # Peacock Pose
        (5, 7, 11),   # RightShoulder, rightElbow, leftHip (to ensure elbows are positioned correctly under the torso)
        (6, 8, 12)    # LeftShoulder, leftElbow, rightHip (to ensure balance and core engagement)
    ],
    'Bhairavasana': [  # Fierce Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to ensure the twist and extension)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to ensure the twist and extension)
    ],
    'Utthita Trikonasana': [  # Extended Triangle Pose
        (5, 11, 15),  # RightShoulder, leftHip, leftAnkle (to ensure the side bend and stretch)
        (6, 12, 16)   # LeftShoulder, rightHip, rightAnkle (to ensure the side bend and stretch)
    ],
    'Supta Padangusthasana': [  # Reclining Hand-to-Big-Toe Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to ensure leg extension and hip opening)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to ensure leg extension and hip opening)
    ],
    'Garudasana': [  # Eagle Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to check the wrapping of the legs)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to check the wrapping of the arms)
    ],
    'Tulasana': [  # Scale Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to ensure lift and core engagement)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to ensure lift and core engagement)
    ],
    'Garbha Pindasana': [  # Embryo in Womb Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to ensure compactness and rounding of the body)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to ensure compactness and rounding of the body)
    ],
    'Eka Pada Rajakapotasana II': [  # One-Legged King Pigeon Pose II
        (5, 11, 15),  # RightShoulder, leftHip, leftAnkle (to check back leg extension and front torso lift)
        (6, 12, 16)   # LeftShoulder, rightHip, rightAnkle (to check back leg extension and front torso lift)
    ],
    'Dwi Pada Viparita Dandasana': [  # Upward Facing Two-Foot Staff Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to ensure backbend and arm positioning)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to ensure backbend and arm positioning)
    ],
    'Kapotasana': [  # Pigeon Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to ensure front leg alignment and back leg extension)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to ensure front leg alignment and back leg extension)
    ],
    'Bhekasana': [  # Frog Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to ensure leg flexion and foot placement)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to ensure leg flexion and foot placement)
    ],
    'Savasana': [  # Corpse Pose
        (5, 11, 12),  # RightShoulder, leftHip, rightHip (to ensure relaxation and alignment)
        (6, 12, 11)   # LeftShoulder, rightHip, leftHip (to ensure relaxation and alignment)
    ],
    'Vriksasana': [  # Tree Pose
        (5, 11, 13),  # RightShoulder, rightHip, rightKnee (to check balance and standing leg stability)
        (6, 12, 14)   # LeftShoulder, leftHip, leftKnee (to check raised leg's position and hip opening)
    ],
    'Krounchasana': [  # Heron Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to ensure leg extension and straightness)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to ensure seated balance and alignment)
    ],
    'Salamba Sarvangasana': [  # Shoulder Stand
        (5, 11, 12),  # RightShoulder, leftHip, rightHip (to ensure shoulder support and body alignment)
        (6, 12, 11)   # LeftShoulder, rightHip, leftHip (to ensure vertical extension and stability)
    ],
    'Astavakrasana': [  # Eight-Angle Pose
        (5, 7, 9),    # RightShoulder, rightElbow, rightWrist (to check arm balance and leg wrapping)
        (6, 8, 10)    # LeftShoulder, leftElbow, leftWrist (to check arm balance and leg wrapping)
    ],
    'Durvasasana': [  # Durvasa's Pose
        (11, 13, 15),  # LeftHip, leftKnee, leftAnkle (to check leg lift and extension behind the head)
        (12, 14, 16)   # RightHip, rightKnee, rightAnkle (to maintain balance and alignment)
    ]
}


def provide_pose_correction_feedback_using_angles(user_keypoints, reference_keypoints, intended_pose_name):
    feedback = []
    angle_keypoints_indices = POSE_ANGLES_KEYPOINTS.get(intended_pose_name, [])

    for indices in angle_keypoints_indices:
        # Extract keypoints for the user
        user_kp1 = user_keypoints[indices[0]][0][:2]  # Get first keypoint's x, y
        user_kp2 = user_keypoints[indices[1]][0][:2]  # Get second keypoint's x, y
        user_kp3 = user_keypoints[indices[2]][0][:2]  # Get third keypoint's x, y

        # Extract keypoints for the reference
        ref_kp1 = reference_keypoints[indices[0]][0][:2]  # Reference first keypoint's x, y
        ref_kp2 = reference_keypoints[indices[1]][0][:2]  # Reference second keypoint's x, y
        ref_kp3 = reference_keypoints[indices[2]][0][:2]  # Reference third keypoint's x, y

        # Calculate angles
        user_angle = calculate_angle(user_kp1, user_kp2, user_kp3)
        ref_angle = calculate_angle(ref_kp1, ref_kp2, ref_kp3)
        
        # Always provide feedback, regardless of angle difference
        feedback.append(f"Pose: {intended_pose_name}, Keypoints: {indices}, User angle: {user_angle:.2f}, Reference angle: {ref_angle:.2f}, Difference: {abs(user_angle - ref_angle):.2f} degrees.")

    return feedback



def handle_user_uploaded_image(user_image_path, intended_pose_name):
    yolo_net, yolo_layers = load_yolo_model()
    user_image = detect_and_crop_person(user_image_path, yolo_net, yolo_layers)
    if user_image is not None:
        print("Person detected and image cropped successfully.")
    else:
        print("No person detected in the image.")
    # user_image = detect_and_crop_person(user_image_path, yolo_net, yolo_layers)
    if user_image is not None:
        posenet_model = load_posenet_model()
        user_image_np = np.array(user_image)  # Convert PIL Image to numpy array if it's not already
        user_image_tf = tf.convert_to_tensor(user_image_np, dtype=tf.float32)
        user_image_tf = preprocess_image_for_posenet(user_image_tf)  # Preprocess the image

        # Correctly invoke the model using its signature and the expected argument name
        
        user_keypoints = posenet_model.signatures['serving_default'](input=user_image_tf)
        keypoints = user_keypoints['output_0'].numpy()[0]  # Adjust indexing based on your model's output
        print("Detected keypoints:", user_keypoints)
        print("it's work")

        # Load reference keypoints from a JSON file
        with open('reference_poses_keypoints.json', 'r') as f:
            reference_poses_keypoints = json.load(f)
        # Ensure pose names in the JSON file are case-insensitive
        intended_pose_name = intended_pose_name.lower()  # Convert intended pose name to lowercase for matching        
        reference_keypoints = reference_poses_keypoints.get(intended_pose_name, [])
        print(reference_keypoints)
        if reference_keypoints:
            correction_feedback = provide_pose_correction_feedback_using_angles(keypoints, reference_keypoints[0], intended_pose_name)
            print(correction_feedback)
            for line in correction_feedback:
                print(line)
            print("Done good pose.")
        else:
            print(f"No reference keypoints available for pose: {intended_pose_name}")
    else:
        print("No person detected in the image.")


# Example usage
user_image_path = r"DATASET\data\adho mukha svanasana\10. screen-shot-2017-09-15-at-17.00.06-1024x585.png"
intended_pose_name = "Adho Mukha Svanasana"
handle_user_uploaded_image(user_image_path, intended_pose_name)
